In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import PauliList, SparsePauliOp
from qiskit_addon_obp import backpropagate
from qiskit_addon_obp.utils.lindblad_noise import PauliLindbladError, PauliLindbladErrorInstruction

In [2]:
lerr = PauliLindbladError(PauliList(["IX"]), [1e-3])
lerr_instr = PauliLindbladErrorInstruction(lerr)

op = SparsePauliOp(["XY", "ZI", "II"], [3, 2, 1])

slices = []

qc_base = QuantumCircuit(2)

qc = qc_base.copy_empty_like()
qc.h(0)
slices.append(qc)

qc = qc_base.copy_empty_like()
qc.cx(0, 1)
qc.append(lerr_instr, [0, 1])
slices.append(qc)

qc_full = qc_base.copy_empty_like()
for sl in slices:
    qc_full.compose(sl, inplace=True)
    qc_full.barrier()
qc_full.draw(
    fold=-1,
)

┌───┐ ░      ┌─────────────┐ ░ 
q_0: ┤ H ├─░───■──┤0            ├─░─
     └───┘ ░ ┌─┴─┐│  Layererror │ ░ 
q_1: ──────░─┤ X ├┤1            ├─░─
           ░ └───┘└─────────────┘ ░

In [3]:
op_prop, remaining_slices, metadata = backpropagate(
    observables=[op],
    slices=slices,
)

In [4]:
op_prop

[SparsePauliOp(['IY', 'II', 'ZX'],
               coeffs=[-2.994006+0.j,  1.      +0.j,  2.      +0.j])]

```
[SparsePauliOp(['IY', 'II', 'ZX'],
               coeffs=[-2.994006+0.j,  1.      +0.j,  1.996004+0.j])]
```

# PROBLEM
This test case returns a slightly different answer now, even when I checkout Samantha's original branch. 
This needs to be understood. 